In [4]:
import csv
import json
import pandas as pd
import numpy as np
import psycopg2

In [3]:
df = pd.read_csv('employee_salaries_1.csv')

In [4]:
df.to_csv('employee_salaries_2.csv', index=False, quoting=csv.QUOTE_ALL)

In [5]:
df['Initial Hire Date'] = pd.to_datetime(df['Initial Hire Date'])

In [6]:
df1 = df[df.Department.isin(['ECC', 'CIT', 'EMS'])]

In [7]:
df2 = df1[df1['Initial Hire Date'].dt.year >= 2010]

In [8]:
df2['Salary'] = df2['Salary'].astype(int)

C:\Users\MacTavish\AppData\Local\Temp\ipykernel_423200\2973334192.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Salary'] = df2['Salary'].astype(int)


In [11]:
df2['Position Title'].sort_values()

701                           311-Operations Supervisor
632                                    Account Clerk II
132                                    Account Clerk II
130                                    Account Clerk II
134                                   Account Clerk III
                             ...                       
233                                Systems Engineer III
174                                Systems Engineer III
268                                Systems Engineer III
259    Telecommunication System Installation Technician
258    Telecommunication System Installation Technician
Name: Position Title, Length: 287, dtype: object

In [ ]:
df2.groupby(by='Department').agg({'Salary': sum})

In [ ]:
reader = CsvReader()
lines = reader.read_csv('employee_salaries_1.csv')

In [ ]:
emp_list = []

In [ ]:
for line in lines[1:]:
    if line[0] in ['ECC', 'CIT', 'EMS']:
        if int(line[5][-4:]) > 2010:
            emp_list.append(Employee.from_csv_line(line))

In [ ]:
line[-1]

In [ ]:
emp_list[0].to_json()

In [ ]:
set([e.hr_date[-4:] for e in emp_list])

In [ ]:
emp = {'dpt': line[0],
        'div': line[1],
        'pos': line[3],
        'hr_date': line[5],
        'salary': int(float(line[7]))}

In [ ]:
json.dumps(emp)

In [ ]:
e = emp_list[0]

In [ ]:
emp

In [ ]:
try:
    conn = psycopg2.connect(
        host="localhost",
        database="postgres",
        user="postgres",
        password="postgres")
    conn.autocommit = True

    # create a cursor
    cur = conn.cursor()
    cur.execute(f"""
                insert into department_employee(
                    department
                    , department_division
                    , position_title
                    , hire_date
                    , salary
                    ) values (
                    '{emp['dpt']}'
                    , '{emp['div']}'
                    , '{emp['pos']}'
                    , '{emp['hr_date']}'
                    , {emp['salary']}
                    ) on conflict do nothing
                """)
    cur.execute(f"""
                insert into department_employee_salary(
                    department
                    , total_salary
                    ) values (
                    '{emp['dpt']}'
                    , {emp['salary']}
                    ) on conflict (department) do update set
                    total_salary = department_employee_salary.total_salary + {emp['salary']}
                """)
    cur.close()

except Exception as e:
    print('error: ', e)

In [7]:
import json
import random
import string
import sys

import psycopg2
from confluent_kafka import Consumer, KafkaError, KafkaException
from confluent_kafka.serialization import StringDeserializer

TOPIC_NAME = "employee_salary"


wait = 0
count = 0
host = "localhost"
port = "29092"

conf = {'bootstrap.servers': f'{host}:{port}',
        'group.id': "employee_salary",
        'enable.auto.commit': True,
        'auto.offset.reset': 'earliest'}
consumer = Consumer(conf)
keep_runnning = True


In [4]:
def persist_employee(msg):
    emp = json.loads(StringDeserializer('utf_8')(msg.value()))
    print(emp)

In [8]:
try:
    consumer.subscribe([TOPIC_NAME])
    while keep_runnning:
        if wait >= 5:
            keep_runnning = False
            continue

        msg = consumer.poll(1.0)
        if msg is None:
            print("Waiting...")
            wait += 1

        elif msg.error():
            if msg.error().code() == KafkaError.PARTITION_EOF:
                # End of partition event
                sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                    (msg.topic(), msg.partition(), msg.offset()))
            elif msg.error():
                raise KafkaException(msg.error())
        else:
            persist_employee(msg)
            count += 1
            print(f'{count} messages have received.')
            break

except KeyboardInterrupt:
    pass

finally:
    consumer.close()

Waiting...
Waiting...
Waiting...
{'dpt': 'CIT', 'div': 'CIT 114 Mail Service', 'pos': 'Distribution Clerk II', 'hr_date': '22-Feb-2015', 'salary': 32405}
1 messages have received.


In [13]:
type(StringDeserializer('utf_8')(msg.value()))

str

In [ ]:
try:
    conn = psycopg2.connect(
        host="localhost",
        database="postgres",
        user="postgres",
        password="postgres")
    conn.autocommit = True

    # create a cursor
    cur = conn.cursor()
    cur.execute(f"""
                insert into department_employee(
                    department
                    , department_division
                    , position_title
                    , hire_date
                    , salary
                    ) values (
                    '{emp['dpt']}'
                    , '{emp['div']}'
                    , '{emp['pos']}'
                    , '{emp['hr_date']}'
                    , {emp['salary']}
                    ) on conflict do nothing
                """)
    cur.execute(f"""
                insert into department_employee_salary(
                    department
                    , total_salary
                    ) values (
                    '{emp['dpt']}'
                    , {emp['salary']}
                    ) on conflict (department) do update set
                    total_salary = department_employee_salary.total_salary + {emp['salary']}
                """)
    cur.close()

except Exception as e:
    print('error: ', e)

In [6]:
def read_cdc(last_id):
    try:
        conn = psycopg2.connect(
            host="localhost",
            port=5433,
            database="source",
            user="source",
            password="source")
        conn.autocommit = True

        cur = conn.cursor()
        cur.execute(f"""
                    select *
                    from employees_cdc
                    where action_id > {last_id}
                    order by action_id
                    """)
        return cur.fetchall()

    except Exception as e:
        print('error: ', e)

In [18]:
a = read_cdc(0)

In [17]:
if a:
    print('1')

In [20]:
for i in a:
    print(type(i[4]))

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>


In [22]:
try:
    conn = psycopg2.connect(
        host="localhost",
        port=5434,
        database="target",
        user="target",
        password="target")
    conn.autocommit = True
    cur = conn.cursor()
    cur.execute(f"""
                select *
                from employees
                """)
    print(cur.fetchall())
    cur.close()
except Exception as e:
    print(emp)
    print('error: ', e)

[]
